In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

import html
from time import sleep

In [6]:
def scrol():
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

def get_pages(driver, url, result, t_sleep=1):
    driver.get(url)
    sleep(t_sleep)
    while True:
        scrol()
        elements = driver.find_elements_by_class_name("uy")
        now_len = len(elements)
        if now_len < 10:
            break
        element = elements[now_len - 1].find_element_by_css_selector('a').get_attribute('href')
        driver.get(element)
        
    links = driver.find_elements_by_class_name("kd")
    autors = driver.find_elements_by_class_name("ld")
 
    for link, autor in zip(links, autors):
        classes = link.get_attribute('class').split()  
        classes_ = autor.get_attribute('class').split() 
        book_info = {
            'link': link.find_element_by_css_selector('a').get_attribute('href'),
            'autor': autor.text,
        }

        result.append(book_info)
        
    return result

In [7]:
url_format = "https://www.bookvoed.ru/author/books?id={}&view=list"

autor_id = ["29369", "29442", "102994", "26149", "35952"]
result = []
with webdriver.Chrome() as driver:
    for autor in autor_id:
        url_current = url_format.format(autor)
        result = get_pages(driver, url_current, result)
        

In [9]:
result = pd.DataFrame(result)
result.head(5)

,autor,link
0,Донцова Дарья Аркадьевна,https://www.bookvoed.ru/book?id=4493310
1,Донцова Дарья Аркадьевна,https://www.bookvoed.ru/book?id=4458002
2,Донцова Дарья Аркадьевна,https://www.bookvoed.ru/book?id=4455170
3,Донцова Дарья Аркадьевна,https://www.bookvoed.ru/book?id=4453205
4,Донцова Дарья Аркадьевна,https://www.bookvoed.ru/book?id=4452365


In [10]:
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import re
import requests


In [11]:
def book_parser(url_current):
    book_html = requests.get(url_current).text
    soup = BeautifulSoup(book_html, 'html.parser')
    book_info = {
        'ID': re.findall('\d+', soup.find('script').text)[0],
        'Название': soup.find('h1', itemprop='name').text.split(':')[0],
        'Рейтинг': soup.find('div', class_="af").attrs['style'].split(':')[1],
        #'Описание': re.sub('\s+', ' ', soup.find('div', class_="lw").text),
        'Обложка': soup.find('img').attrs['src']

    }
    if (soup.find('div', class_="ov nM") is not None):
        book_info['Возраст'] = '0+'
        
    if (soup.find('div', class_="pv nM") is not None):
        book_info['Возраст'] = '6+'
    
    if (soup.find('div', class_="rv nM") is not None):
        book_info['Возраст'] = '16+'
    
    if (soup.find('div', class_="qv nM") is not None):
        book_info['Возраст'] = '12+'
    
    if (soup.find('a', class_="Ke Me ") is None or len(soup.find('a', class_="Ke Me ").text.split()) == 0):
        book_info['Понравилось'] = 0
    else:
        book_info['Понравилось'] = soup.find('a', class_="Ke Me ").text.split()[0]
    
    if (soup.find('a', class_="Ke Oe ") is None or len(soup.find('a', class_="Ke Oe ").text.split()) == 0):
        book_info['Не понравилось'] = 0
    else:
        book_info['Не понравилось'] = soup.find('a', class_="Ke Oe ").text.split()[0]
    
    if (soup.find('a', class_="Ke Le ") is None or len(soup.find('a', class_="Ke Le ").text.split()) == 0):
        book_info['В закладки'] = 0
    else:
        book_info['В закладки'] = soup.find('a', class_="Ke Le ").text.split()[0]
    
    if (soup.find('meta', itemprop='price') is None):
        book_info['Цена'] = ''
    else:
        book_info['Цена'] = soup.find('meta', itemprop='price').attrs['content']
        
    if (soup.find('div', class_="lw") is None):
        book_info['Описание'] = ''
    else:
        book_info['Описание'] = re.sub('\s+', ' ', soup.find('div', class_="lw").text)

    table = soup.find('table', class_='tw')
    rows = table.find_all('tr')

    for row in rows:
        for script in row.find_all("script"):
            script.replace_with('')
        
    data = [list(map(lambda x: re.sub("\s+", ' ', x.text.strip()), row.find_all('td')))
            for row in rows]
    for d in data:
        book_info[d[0]] = d[1]
    
    return book_info



In [12]:
result = list(map(book_parser, result['link']))
    
df = pd.DataFrame(result)
df.sort_values(by=['ID'], inplace=True)


In [15]:
with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)